<a href="https://colab.research.google.com/github/visumania/DeepSexist/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Montura de google colab para poder leer/escribir en Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importación de librerías necesarias

In [2]:
import pandas as pd

Lectura y tratamiento de los datos del fichero

In [4]:
df = pd.read_json('/content/drive/MyDrive/EXIST 2025 Videos Dataset/training/EXIST2025_training.json', orient='index') # Lectura del dataset desde Drive
df.reset_index(drop=True, inplace=True) # Reseteamos el index del dataframe
df = df.drop(columns=["id_Tiktok", "id_EXIST", "video", "url", "annotators", "gender_annotators", "labels_task3_2", "split"]) # Eliminación de columnas que no son relevantes del dataframe

In [5]:
# Distribución de datos importantes en el fichero

print("Distribución del número de anotadores")
print(df.value_counts("number_annotators"))

print("\nDistribución del idioma")
print(df.value_counts("lang"))

print("\nDistribución del etiquetado para la tarea 1")
print(df['labels_task3_1'].value_counts())

print("\nDistribución del etiquetado para la tarea 3")
print(df['labels_task3_3'].value_counts())

Distribución del número de anotadores
number_annotators
2    1780
3     744
Name: count, dtype: int64

Distribución del idioma
lang
es    1524
en    1000
Name: count, dtype: int64

Distribución del etiquetado para la tarea 1
labels_task3_1
[NO, NO]               1069
[YES, YES]              711
[YES, NO, YES]          255
[NO, YES, NO]           140
[YES, YES, YES]         108
[NO, YES, YES]          101
[YES, NO, NO]            62
[UNKNOWN, NO, NO]        20
[UNKNOWN, YES, YES]      19
[NO, UNKNOWN, NO]        15
[UNKNOWN, YES, NO]        7
[UNKNOWN, NO, YES]        6
[YES, UNKNOWN, YES]       4
[YES, YES, NO]            4
[YES, UNKNOWN, NO]        2
[NO, UNKNOWN, YES]        1
Name: count, dtype: int64

Distribución del etiquetado para la tarea 3
labels_task3_3
[[-], [-]]                                                                                                               1069
[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMINANCE]]                                                

In [32]:
# División del fichero original en 2: uno para la subtarea 3.1 y otro parala subtarea 3.3

df_3_1 = df.drop(columns=["labels_task3_3"])
df_3_3 = df.drop(columns=["labels_task3_1"])

# Expande la columna labels_task3_1 en varias columnas
df_3_1_expanded = df_3_1["labels_task3_1"].apply(pd.Series)

# Renombra las columnas resultantes para mayor claridad
df_3_1_expanded.columns = [f"labels_task_3_1_{i+1}" for i in range(df_3_1_expanded.shape[1])]

# Une con el dataframe original sin la columna original
df_3_1 = pd.concat([df_3_1.drop(columns=["labels_task3_1"]), df_3_1_expanded], axis=1)

# Pasamos a binario los valores YES y NO
df_3_1.iloc[:, 4:] = df_3_1.iloc[:, 4:].replace({"YES": int(1), "NO": int(0)})

# # Esto se hace si queremos determinar si un valor específico de una fila/columna de un dataframe es NaN
# # print(pd.isna(df_3_1.at[0, 'labels_task_3_1_3']))

df_3_1.iloc[:, 6] = df_3_1.iloc[:, 6].astype("Int64")  # Pasamos los valores binarios de la última columna a int64 --> Tipo entero que soporta NaN

# print(str(type(df_3_1.iloc[0,6]))=="<class 'pandas._libs.missing.NAType'>")

print(df_3_1['labels_task_3_1_1'].value_counts())
print(df_3_1['labels_task_3_1_2'].value_counts())
print(df_3_1['labels_task_3_1_3'].value_counts())

# df_3_1

labels_task_3_1_1
0          1326
1          1146
UNKNOWN      52
Name: count, dtype: int64
labels_task_3_1_2
0          1412
1          1090
UNKNOWN      22
Name: count, dtype: int64
labels_task_3_1_3
1    494
0    250
Name: count, dtype: int64


<ipython-input-32-5d2441c2cff3>:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_3_1.iloc[:, 4:] = df_3_1.iloc[:, 4:].replace({"YES": int(1), "NO": int(0)})


Unificado de las anotaciones de los diferentes anotadores/as por mayoría: Si tiene 3 anotadores se unifica en el etiquetado que haya predominado en cada anotador/a.
Si hay 2 anotadores/as y conflictos habrá que decidir si dejarlo como 0 o 1

In [34]:
df_3_1["label_task_3_1_merged"] = 0 # Creación de una nueva columna en donde recogeremos la unión de la etiquetas impuestas por los/las anotadores/as
# df_3_1

In [28]:
for i in range(df_3_1.shape[0]):
  votacionPositiva = votacionNegativa = 0
  numeroAnotadores = df_3_1.iloc[i]['number_annotators']

  if df_3_1.iloc[i]['labels_task_3_1_1'] == 0:
    votacionNegativa += 1
  elif df_3_1.iloc[i]['labels_task_3_1_1'] == 1:
    votacionPositiva += 1

  if df_3_1.iloc[i]['labels_task_3_1_2'] == 0:
    votacionNegativa += 1
  elif df_3_1.iloc[i]['labels_task_3_1_2'] == 1:
    votacionPositiva += 1

  if df_3_1.iloc[i]['labels_task_3_1_1'] == 0:
    votacionNegativa += 1
  elif df_3_1.iloc[i]['labels_task_3_1_1'] == 1:
    votacionPositiva += 1

2524



































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































